<a href="https://colab.research.google.com/github/Mahsa33/SeatingArrangment/blob/main/SeatingArrangment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [2]:
import random

first_names = ["Ali", "Zahra", "Reza", "Sara", "Mohammad", "Fatemeh", "Hossein", "Maryam", "Mehdi", "Narges", "Hamed", "Roya"]
last_names = ["Ahmadi", "Hosseini", "Karimi", "Rahimi", "Hashemi", "Ebrahimi", "Moradi", "Mohammadi", "Rostami", "Fazeli", "Hosseinzadeh", "Niknam"]

names = []
for i in range(24):
    first_name = random.choice(first_names)
    last_name = random.choice(last_names)
    names.append(f"{first_name} {last_name}")


print(names)

['Narges Mohammadi', 'Hossein Rostami', 'Reza Rostami', 'Sara Hashemi', 'Maryam Niknam', 'Zahra Hashemi', 'Narges Niknam', 'Roya Hosseinzadeh', 'Sara Ahmadi', 'Maryam Rahimi', 'Zahra Ahmadi', 'Hamed Mohammadi', 'Roya Hashemi', 'Hamed Rahimi', 'Narges Fazeli', 'Hossein Ahmadi', 'Mohammad Moradi', 'Ali Ahmadi', 'Fatemeh Moradi', 'Narges Fazeli', 'Maryam Fazeli', 'Hamed Rahimi', 'Sara Mohammadi', 'Hamed Fazeli']


In [3]:
matrix_c = [[0 for x in range(24)] for y in range(24)]

count_one = 0
while True:
    i = random.randint(0, 23)
    j = random.randint(0, 23)
    if i != j:
      if matrix_c[i][j] == 0:
          matrix_c[i][j] = 1
          count_one += 1
          if count_one == 40:
            break


In [4]:
# Convert the matrix to a NumPy array
matrix_np = np.array(matrix_c)

# Convert the NumPy array to a PyTorch tensor
conflict_matrix = torch.from_numpy(matrix_np)

In [12]:
num_people =24

class SeatingNet(nn.Module):
    def __init__(self):
        super().__init__()
        super(SeatingNet, self).__init__()
        self.fc1 = nn.Linear(num_people, 128)
        self.fc4 = nn.Linear(128, num_people)

    def forward(self, x):
        out = self.fc1(x)
        out = torch.relu(out)
        out = self.fc4(out)
        return out


model = SeatingNet()


def loss(output, conflict_matrix):
    # محاسبه احتمال نشستن هر نفر در هر صندلی
    probabilities = torch.softmax(output, dim=1)
    probabilities = probabilities.reshape(24)

    # محاسبه جریمه برای زوج‌های ناسازگار
    penalty = 0
    for i in range(num_people-1):
        for j in range(i + 1, num_people-1):
            if conflict_matrix[i, j] == 1:
                penalty += probabilities[i] * probabilities[j]


    return penalty

optimizer = optim.Adam(model.parameters())

In [32]:
import random
import torch

class SeatingArrangement:
    def __init__(self, num_people=24, rows=6, cols=4, conflict_matrix = conflict_matrix):
        """
        Initializes the SeatingArrangement class.

        Args:
            num_people (int, optional): The total number of people. Defaults to 24.
            rows (int, optional): The number of rows in the seating arrangement. Defaults to 6.
            cols (int, optional): The number of columns in the seating arrangement. Defaults to 4.
        """
        self.num_people = num_people
        self.rows = rows
        self.cols = cols
        self.conflict_matrix = conflict_matrix

    def generate_seating_order(self):
        """
        Generates a random seating order with the specified number of people, rows, and columns.

        Returns:
            torch.FloatTensor: A 2D tensor representing the seating order, where
                              each element corresponds to a person's seat number.
        """
        person_ids = [i for i in range(self.num_people)]
        random.shuffle(person_ids)

        seating_order = torch.zeros(self.rows, self.cols)
        k = 0
        for i in range(self.rows):
            for j in range(self.cols):
                seating_order[i, j] = person_ids[k]
                k += 1

        return seating_order.float()

    def analyze_seating_order_conflicts(self, seating_order, conflict_matrix):
        """
        Analyzes the seating order and detects conflicts based on the provided conflict matrix.

        Args:
            seating_order (torch.FloatTensor): A 2D tensor representing the seating order.
            conflict_matrix (torch.Tensor): A 2D tensor representing the conflict matrix,
                                            where 1 indicates a conflict between two people.

        Returns:
            torch.Tensor: A 2D tensor representing the conflicts in the seating order.
        """
        seating_order_conflicts = torch.zeros(self.rows, self.cols)

        # Iterate over the seating order and check for conflicts
        for i in range(self.rows):
            for j in range(self.cols):
                person = int(seating_order[i, j])

                # Check conflicts with adjacent persons in the same row
                if j > 0:
                    seating_order_conflicts[i, j] += conflict_matrix[person, int(seating_order[i, j - 1])]
                if j < self.cols - 1:
                    seating_order_conflicts[i, j] += conflict_matrix[person, int(seating_order[i, j + 1])]

                # Check conflicts with adjacent persons in the same column
                if i > 0:
                    seating_order_conflicts[i, j] += conflict_matrix[person, int(seating_order[i - 1, j])]
                if i < self.rows - 1:
                    seating_order_conflicts[i, j] += conflict_matrix[person, int(seating_order[i + 1, j])]

        return seating_order_conflicts

    def find_incompatible_pairs(self, seating_order_conflicts):
        """
        Finds the incompatible pairs in the seating order conflicts.

        Args:
            seating_order_conflicts (torch.Tensor): A 2D tensor representing the conflicts in the seating order.

        Returns:
            list: A list of incompatible pairs.
        """
        incompatible_pairs = []
        for i in range(seating_order_conflicts.shape[0]):
            for j in range(seating_order_conflicts.shape[1]):
                if seating_order_conflicts[i, j] != 0:
                    incompatible_pairs.append((i, j))
        return incompatible_pairs

    def get_conflicted_seats(self, seating_order, incompatible_pairs):
        """
        Gets the conflicted seats in the seating order.

        Args:
            seating_order (torch.FloatTensor): A 2D tensor representing the seating order.
            incompatible_pairs (list): A list of incompatible pairs.

        Returns:
            list: A list of conflicted seat numbers.
        """
        seat_conflict = []
        for i, j in incompatible_pairs:
            seat_conflict.append(int(seating_order[i][j]))
        return seat_conflict

    def swap_conflict_seats(self, incompatible_pairs, seat_conflict, seating_order):
        """
        Swaps the conflicted seats in the seating order.

        Args:
            incompatible_pairs (list): A list of incompatible pairs.
            seat_conflict (list): A list of conflicted seat numbers.
            seating_order (torch.FloatTensor): A 2D tensor representing the seating order.

        Returns:
            torch.FloatTensor: The updated seating order.
        """
        for coord, value in zip(incompatible_pairs, seat_conflict):
            i, j = coord
            seating_order[i, j] = value
        return seating_order

    def update_conflict_new_order(self, seating_order, conflict_matrix):
        """
        Updates the conflicts in the seating order.

        Args:
            seating_order (torch.FloatTensor): A 2D tensor representing the seating order.
            conflict_matrix (torch.Tensor): A 2D tensor representing the conflict matrix.

        Returns:
            torch.Tensor: The updated seating order conflicts.
        """
        update_matrix = self.analyze_seating_order_conflicts(seating_order, conflict_matrix)
        return update_matrix

In [51]:
rows , cols = 6, 4
loss_value = 0

seating_arrangement = SeatingArrangement(num_people=24, rows=6, cols=4, conflict_matrix=conflict_matrix)
seating_order = seating_arrangement.generate_seating_order()
seating_order_conflicts = seating_arrangement.analyze_seating_order_conflicts(seating_order, conflict_matrix)
print('seating_order:', seating_order)
print('seating_order_conflicts:', seating_order_conflicts)


for epoch in range(100):

    if epoch != 0 :
      incompatible_pairs = seating_arrangement.find_incompatible_pairs(seating_order_conflicts)
      print('find incompatible pairs using conflict matrix:', incompatible_pairs)
      conflicted_seats  = seating_arrangement.get_conflicted_seats(seating_order, incompatible_pairs)
      print('find incompatible seat numbers', conflicted_seats)

      if len(conflicted_seats) == 1:
        break

      random.shuffle(conflicted_seats)
      new_seating_order = seating_arrangement.swap_conflict_seats(incompatible_pairs, conflicted_seats, seating_order)
      print('new generated seating order:', new_seating_order)
      updated_conflict = seating_arrangement.update_conflict_new_order(seating_order, conflict_matrix)
      print('update conflict seating order:', updated_conflict)

      if len(conflicted_seats) == 2 and torch.sum(updated_conflict == 1) == 2:
        if loss_value < 1.000000e-06:
          break


      seating_order = new_seating_order
      seating_order_conflicts = updated_conflict

    # پیش‌بینی احتمال نشستن برای هر نفر
    output = model(torch.FloatTensor(seating_order_conflicts).view(-1, rows*cols))


    loss_value = loss(output, conflict_matrix)
    prev_loss = loss_value

    # صفر کردن گرادیان‌ها
    optimizer.zero_grad()

    # محاسبه گرادیان‌ها
    loss_value.backward()

    # به‌روزرسانی پارامترهای مدل
    optimizer.step()

    # نمایش loss
    print(f"Epoch {epoch + 1}: loss = {loss_value.item():.7}")

seating_order: tensor([[11.,  3.,  4.,  1.],
        [ 2.,  0.,  9., 14.],
        [18., 23., 22.,  6.],
        [20., 16., 13., 10.],
        [15.,  8., 12.,  7.],
        [17., 19.,  5., 21.]])
seating_order_conflicts: tensor([[0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [2., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.]])
Epoch 1: loss = 4.979061e-06
find incompatible pairs using conflict matrix: [(1, 0), (2, 0), (4, 0)]
find incompatible seat numbers [2, 18, 15]
new generated seating order: tensor([[11.,  3.,  4.,  1.],
        [15.,  0.,  9., 14.],
        [ 2., 23., 22.,  6.],
        [20., 16., 13., 10.],
        [18.,  8., 12.,  7.],
        [17., 19.,  5., 21.]])
update conflict seating order: tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.]])
Epoch 2: loss = 1.965013e-05
find incompatible pairs using conflict matrix: [(2,